## Content-Based Recommender System
### Item based recommendation using knn

Source: https://towardsdatascience.com

In [175]:
##Dataset url: https://grouplens.org/datasets/movielens/latest/

import pandas as pd
import numpy as np

In [176]:
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [177]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [178]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.000
1,1,3,4.000
2,1,6,4.000
3,1,47,5.000
4,1,50,5.000


In [179]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.000,Toy Story (1995)
1,5,1,4.000,Toy Story (1995)
2,7,1,4.500,Toy Story (1995)
3,15,1,2.500,Toy Story (1995)
4,17,1,4.500,Toy Story (1995)


In [180]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (combine_movie_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()


,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [181]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [182]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [183]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [184]:
rating_popular_movie.shape

(41362, 5)

In [185]:
## First lets create a Pivot matrix

movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [186]:
movie_features_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X2: X-Men United (2003),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,4.000
You've Got Mail (1998),0.000,0.000,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,2.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000
Young Frankenstein (1974),5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,3.500,0.000,0.000,0.000,0.000


In [187]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [188]:
movie_features_df.shape

(450, 606)

In [189]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)


231


In [190]:
indices

array([[231, 232, 357, 154, 308,  44]], dtype=int64)

In [191]:
distances

array([[5.9604645e-08, 1.4694804e-01, 3.3118910e-01, 3.6026245e-01,
        3.6622548e-01, 3.8345039e-01]], dtype=float32)

In [192]:
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [193]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Kill Bill: Vol. 1 (2003):

1: Kill Bill: Vol. 2 (2004), with distance of 0.14694803953170776:
2: Sin City (2005), with distance of 0.3311890959739685:
3: Fight Club (1999), with distance of 0.3602624535560608:
4: Pirates of the Caribbean: The Curse of the Black Pearl (2003), with distance of 0.3662254810333252:
5: Batman Begins (2005), with distance of 0.38345038890838623:


# Collaborative Filtering
### (User Based)

In [194]:
movie_features_df.transpose().head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,...,5.000,5.000,0.000,5.000,0.000,0.000,0.000,5.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.500,0.000,0.000,0.000
4,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,4.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [195]:
movie_features_df = movie_features_df.transpose()
movie_features_df_matrix = csr_matrix(movie_features_df)

In [196]:
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [197]:
# query_index = np.random.choice(movie_features_df.shape[0])
#query_index = 375
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)


231


In [198]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 234:

1: 186, with distance of 0.4768962264060974:
2: 217, with distance of 0.48978400230407715:
3: 19, with distance of 0.5110403895378113:
4: 380, with distance of 0.5632767677307129:
5: 27, with distance of 0.5685693025588989:


In [199]:
movie_features_df.iloc[indices.flatten(), :]

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
234,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,...,4.000,4.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000
186,0.000,0.000,5.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,...,5.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
217,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,3.000,1.000,...,3.000,3.000,0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000
19,3.000,0.000,3.000,0.000,0.000,0.000,0.000,3.000,2.000,2.000,...,4.000,0.000,0.000,4.000,0.000,0.000,2.000,4.000,0.000,0.000
380,0.000,0.000,4.000,3.000,4.000,0.000,3.000,4.000,5.000,5.000,...,4.000,3.000,0.000,5.000,4.000,5.000,0.000,5.000,3.000,0.000
27,0.000,0.000,2.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [200]:
not_watched_movie_list = movie_features_df.iloc[indices.flatten()[0], :][movie_features_df.iloc[indices.flatten()[0], :] == 0.0].index
not_watched_movie_list

Index(['10 Things I Hate About You (1999)', '12 Angry Men (1957)',
       '2001: A Space Odyssey (1968)', '28 Days Later (2002)', '300 (2007)',
       '40-Year-Old Virgin, The (2005)', 'A.I. Artificial Intelligence (2001)',
       'Abyss, The (1989)', 'Ace Ventura: When Nature Calls (1995)',
       'Addams Family Values (1993)',
       ...
       'When Harry Met Sally... (1989)', 'While You Were Sleeping (1995)',
       'Wild Wild West (1999)', 'Wolf of Wall Street, The (2013)',
       'X-Men: The Last Stand (2006)', 'X2: X-Men United (2003)',
       'You've Got Mail (1998)', 'Young Frankenstein (1974)',
       'Zombieland (2009)', 'Zoolander (2001)'],
      dtype='object', name='title', length=372)

In [201]:
movie_features_df.iloc[indices.flatten()][not_watched_movie_list]

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),...,When Harry Met Sally... (1989),While You Were Sleeping (1995),Wild Wild West (1999),"Wolf of Wall Street, The (2013)",X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
234,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
186,0.000,0.000,5.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,...,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
217,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,1.000,3.000,...,0.000,0.000,0.000,0.000,0.000,0.000,3.000,3.000,0.000,0.000
19,3.000,0.000,3.000,0.000,0.000,0.000,0.000,3.000,2.000,2.000,...,3.000,3.000,1.000,0.000,0.000,0.000,2.000,4.000,0.000,0.000
380,0.000,0.000,4.000,3.000,4.000,0.000,3.000,4.000,5.000,3.000,...,0.000,0.000,4.000,0.000,4.000,5.000,0.000,5.000,3.000,0.000
27,0.000,0.000,2.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [203]:
movie_features_df[not_watched_movie_list].columns

Index(['10 Things I Hate About You (1999)', '12 Angry Men (1957)',
       '2001: A Space Odyssey (1968)', '28 Days Later (2002)', '300 (2007)',
       '40-Year-Old Virgin, The (2005)', 'A.I. Artificial Intelligence (2001)',
       'Abyss, The (1989)', 'Ace Ventura: When Nature Calls (1995)',
       'Addams Family Values (1993)',
       ...
       'When Harry Met Sally... (1989)', 'While You Were Sleeping (1995)',
       'Wild Wild West (1999)', 'Wolf of Wall Street, The (2013)',
       'X-Men: The Last Stand (2006)', 'X2: X-Men United (2003)',
       'You've Got Mail (1998)', 'Young Frankenstein (1974)',
       'Zombieland (2009)', 'Zoolander (2001)'],
      dtype='object', name='title', length=372)

In [204]:
movie_features_df[not_watched_movie_list].index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
           dtype='int64', name='userId', length=606)

In [205]:
# recommended_movie_index = np.sort(movie_features_df[not_watched_movie_list].apply(lambda  x: (x!=0).sum(), axis=0).values)[::-1][:5]
recommended_movie_index = np.sort(movie_features_df.iloc[indices.flatten()][not_watched_movie_list].apply(lambda  x: x.sum(), axis=0))

mdf = pd.DataFrame(
              {"movie_count" : recommended_movie_index,
               "title" : movie_features_df[not_watched_movie_list].columns})
mdf

,movie_count,title
0,0.000,10 Things I Hate About You (1999)
1,0.000,12 Angry Men (1957)
2,0.000,2001: A Space Odyssey (1968)
3,0.000,28 Days Later (2002)
4,0.000,300 (2007)
...,...,...
367,18.000,X2: X-Men United (2003)
368,18.000,You've Got Mail (1998)
369,18.000,Young Frankenstein (1974)
370,19.000,Zombieland (2009)


In [206]:
mdf.sort_values(by="movie_count", ascending=False).head()

,movie_count,title
371,22.000,Zoolander (2001)
370,19.000,Zombieland (2009)
369,18.000,Young Frankenstein (1974)
368,18.000,You've Got Mail (1998)
367,18.000,X2: X-Men United (2003)
